<a href="https://colab.research.google.com/github/chiranjeet14/ML_Journey/blob/master/Hackerearth-Predict-the-genetic-disorders/3_genetic_testing_targets_merged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import io
import gc
import time
from pprint import pprint
from datetime import date

# settings
import warnings
warnings.filterwarnings("ignore")
gc.enable()

In [ ]:
# !pip3 install pandas-profiling > /dev/null
# !pip3 install tune-sklearn ray[tune] > /dev/null
!pip3 install imbalanced-learn > /dev/null
# !pip3 install lightgbm > /dev/null
!pip3 install catboost > /dev/null
# !pip3 install xgboost > /dev/null

In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
gDrivePath = '/content/drive/MyDrive/Datasets/Hackerearth_genetic_testing/dataset/'

In [ ]:
df_train = pd.read_csv(gDrivePath+'train_preprocessed.csv')
df_test = pd.read_csv(gDrivePath+'test_preprocessed.csv')

In [ ]:
df_train.sample(3)

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Autopsy shows birth defect (if applicable),Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass
11736,1.0,1.0,0.0,0.604455,0.0,5.103589,34.577116,41.916199,1.0,1.506271,2.0,2.0,1.0,4.0,4.0,2.0,2.0,3.0,4.0,1.0,2.000000,2.000000,1.0,4.387931,1.0,0.0,1.000000,0.0,1.0,0.0,Mitochondrial genetic inheritance disorders,Mitochondrial myopathy
16188,8.0,0.0,0.0,0.000000,0.0,4.777974,50.000000,41.911825,0.0,1.000000,1.0,2.0,3.0,2.0,3.0,2.0,1.0,4.0,2.0,2.0,1.495193,0.739342,2.0,4.300135,2.0,1.0,0.000000,0.0,1.0,0.0,Single-gene inheritance diseases,Tay-Sachs
7460,5.0,0.0,1.0,1.000000,1.0,4.945858,34.579897,40.000000,1.0,2.000000,1.0,1.0,1.0,4.0,4.0,2.0,2.0,3.0,0.0,2.0,2.000000,1.000000,1.0,5.450959,2.0,1.0,0.564098,1.0,0.0,0.0,Mitochondrial genetic inheritance disorders,Leigh syndrome


In [ ]:
df_train['GeneticDisorder-DisorderSubclass'] = df_train['Genetic Disorder'] + '<->' + df_train['Disorder Subclass']

In [ ]:
df_train.sample(3)

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Autopsy shows birth defect (if applicable),Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass,GeneticDisorder-DisorderSubclass
15312,4.0,0.0,1.0,1.0,1.0,5.020725,24.000000,33.000000,1.0,1.0,1.0,2.0,3.0,2.000000,1.0,1.00000,2.0,4.0,2.000000,1.0,2.0,1.00000,2.0,8.584695,2.0,0.0,1.000000,1.0,1.0,1.0,Single-gene inheritance diseases,Cystic fibrosis,Single-gene inheritance diseases<->Cystic fibr...
4113,12.0,0.0,1.0,0.0,0.0,5.167144,34.586108,41.908038,0.0,2.0,1.0,1.0,3.0,2.509599,3.0,1.46276,1.0,4.0,2.229998,1.0,2.0,1.43124,1.0,12.000000,2.0,1.0,0.554179,1.0,0.0,0.0,Single-gene inheritance diseases,Tay-Sachs,Single-gene inheritance diseases<->Tay-Sachs
2777,10.0,1.0,0.0,1.0,1.0,4.581385,23.000000,27.000000,1.0,1.0,2.0,1.0,3.0,4.000000,2.0,2.00000,2.0,4.0,3.000000,2.0,2.0,4.00000,2.0,5.731035,1.0,1.0,1.000000,1.0,1.0,0.0,Single-gene inheritance diseases,Cystic fibrosis,Single-gene inheritance diseases<->Cystic fibr...


In [ ]:
# df_train[['Col1','Col2']] = df_train['GeneticDisorder-DisorderSubclass'].str.split("<->",expand=True)

In [ ]:
# df_train.sample(3)

In [ ]:
# from pandas_profiling import ProfileReport

# df_target = df_train[['Genetic Disorder', 'Disorder Subclass', 'GeneticDisorder-DisorderSubclass']]

# report = ProfileReport(df_target, title="Pandas Profiling Report", explorative=True)
# report.to_file("genetic_testing.html")

In [ ]:
# report

### Checking if the dataset is balanced/imbalanced

In [ ]:
target_count = df_train['GeneticDisorder-DisorderSubclass'].value_counts()
target_count

Mitochondrial genetic inheritance disorders<->Leigh syndrome                         4683
Mitochondrial genetic inheritance disorders<->Mitochondrial myopathy                 3971
Single-gene inheritance diseases<->Cystic fibrosis                                   3145
Single-gene inheritance diseases<->Tay-Sachs                                         2556
Multifactorial genetic inheritance disorders<->Diabetes                              1653
Single-gene inheritance diseases<->Hemochromatosis                                   1228
Mitochondrial genetic inheritance disorders<->Leber's hereditary optic neuropathy     587
Multifactorial genetic inheritance disorders<->Alzheimer's                            133
Multifactorial genetic inheritance disorders<->Cancer                                  91
Name: GeneticDisorder-DisorderSubclass, dtype: int64

### Splitting Data into train-cv

In [ ]:
target_labels = df_train['GeneticDisorder-DisorderSubclass'].values

df_train.drop(['Genetic Disorder','Disorder Subclass', 'GeneticDisorder-DisorderSubclass'], axis=1, inplace=True)
df_test.drop(['Genetic Disorder','Disorder Subclass', 'GeneticDisorder-DisorderSubclass'], axis=1, inplace=True, errors='ignore')

In [ ]:
# classification split for genetic_disorder_labels
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(df_train, target_labels, test_size=0.1, random_state=50)

### Over Sampling using SMOTE

In [ ]:
# https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/
from imblearn.over_sampling import SMOTE
smote_overSampling = SMOTE()
X_train,y_train = smote_overSampling.fit_resample(X_train,y_train)
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{"Mitochondrial genetic inheritance disorders<->Leber's hereditary optic neuropathy": 4192,
 'Mitochondrial genetic inheritance disorders<->Leigh syndrome': 4192,
 'Mitochondrial genetic inheritance disorders<->Mitochondrial myopathy': 4192,
 "Multifactorial genetic inheritance disorders<->Alzheimer's": 4192,
 'Multifactorial genetic inheritance disorders<->Cancer': 4192,
 'Multifactorial genetic inheritance disorders<->Diabetes': 4192,
 'Single-gene inheritance diseases<->Cystic fibrosis': 4192,
 'Single-gene inheritance diseases<->Hemochromatosis': 4192,
 'Single-gene inheritance diseases<->Tay-Sachs': 4192}

### Scaling data

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_cv_scaled = scaler.transform(X_cv)

X_test_scaled = scaler.transform(df_test)

# X_train_scaled

### Modelling & Cross-Validation

In [ ]:
# # %%time
# # # Train multiple models : https://www.kaggle.com/tflare/testing-multiple-models-with-scikit-learn-0-79425
# from sklearn.ensemble import GradientBoostingClassifier
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier


# from sklearn.model_selection import cross_val_score

# models = []

# GradientBoostingClassifier = GradientBoostingClassifier()
# XGBClassifier = XGBClassifier()
# LGBMClassifier = LGBMClassifier()
# CatBoostClassifier = CatBoostClassifier()


# # models.append(("GradientBoostingClassifier",GradientBoostingClassifier))
# # models.append(("XGBClassifier",XGBClassifier))
# models.append(("LGBMClassifier",LGBMClassifier))
# models.append(("CatBoostClassifier",CatBoostClassifier))

# # metric_names = ['f1', 'average_precision', 'accuracy', 'precision', 'recall']
# metric_names = ['f1_weighted']
# results = []
# names = []

# nested_dict = {}

# for name,model in models:
#   nested_dict[name] = {}
#   for metric in metric_names:
#     print("\nRunning : {}, with metric : {}".format(name, metric))
#     score = cross_val_score(model, X_train_scaled, y_train, n_jobs=-1, scoring=metric, cv=5, verbose=1)
#     nested_dict[name][metric] = score.mean()

In [ ]:
# import json
# print(json.dumps(nested_dict, sort_keys=True, indent=4))

## Hyperparameter tuning

### ray tune : ExtraTreesClassifier

In [ ]:
# # from sklearn.model_selection import GridSearchCV
# from tune_sklearn import TuneGridSearchCV

# from sklearn.ensemble import ExtraTreesClassifier

# # model_classifier = ExtraTreesClassifier(max_depth=15, n_estimators=400)
# model_classifier = ExtraTreesClassifier(bootstrap=False, max_features='log2', warm_start=False, max_depth=15, n_estimators=700)

# # # Best Params: Best Params: {'criterion': 'entropy', 'bootstrap': False, 'max_features': 'log2', 'warm_start': False}

# # Parameters to tune:
# parameters = {
#     # 'n_estimators': np.arange(300, 800, 100, dtype=int),
#     # 'max_depth': np.arange(5, 16, 1, dtype=int),
#     'criterion': ['gini', 'entropy'],
#     # 'bootstrap': [True, False],
#     # 'max_features': ['auto', 'sqrt', 'log2'],
#     # 'warm_start': [True, False],
# }
# tune_search_model = TuneGridSearchCV(
#     model_classifier,
#     parameters,
#     scoring='f1_weighted',
#     verbose=1,
#     n_jobs=-1
# )

In [ ]:
# tune_search_model.fit(X_train_scaled, y_train)

In [ ]:
# pred = tune_search_model.predict(X_cv_scaled)
# accuracy = np.count_nonzero(np.array(pred) == np.array(y_cv)) / len(pred)
# print("Tune Accuracy:", accuracy)

In [ ]:
# print("Best Params:", tune_search_model.best_params_)

In [ ]:
# CatBoostClassifier.fit(X_train_scaled, y_train)

In [ ]:
# pred = CatBoostClassifier.predict(X_cv_scaled)
# accuracy = np.count_nonzero(np.array(pred) == np.array(y_cv)) / len(pred)
# print("Accuracy:", accuracy)

### Hyperparameter tuning Catboost

In [ ]:
%%time

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from catboost import CatBoostClassifier

CBC = CatBoostClassifier(loss_function='MultiClass', use_best_model=True, l2_leaf_reg=1, iterations=700, depth=10, border_count=20, task_type='GPU')

parameters = {
    'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3],
    'bagging_temperature':[0.03,0.09,0.25,0.75],
    'random_strength':[0.2,0.5,0.8],
    'max_ctr_complexity':[1,2,3,4,5]
}

Grid_CBC = RandomizedSearchCV(estimator=CBC, param_distributions=parameters, verbose=10, n_iter=100, scoring='f1_weighted')
Grid_CBC.fit(X_train_scaled, y_train, eval_set=(X_cv_scaled, y_cv))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] random_strength=0.8, max_ctr_complexity=3, learning_rate=0.1, bagging_temperature=0.09 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 1.9914955	test: 2.0958760	best: 2.0958760 (0)	total: 80.1ms	remaining: 56s
1:	learn: 1.8474925	test: 2.0113860	best: 2.0113860 (1)	total: 157ms	remaining: 54.8s
2:	learn: 1.7355364	test: 1.9460089	best: 1.9460089 (2)	total: 249ms	remaining: 57.7s
3:	learn: 1.6379742	test: 1.8838973	best: 1.8838973 (3)	total: 325ms	remaining: 56.6s
4:	learn: 1.5546651	test: 1.8245224	best: 1.8245224 (4)	total: 394ms	remaining: 54.8s
5:	learn: 1.4806304	test: 1.7610592	best: 1.7610592 (5)	total: 464ms	remaining: 53.6s
6:	learn: 1.4186547	test: 1.7222432	best: 1.7222432 (6)	total: 553ms	remaining: 54.7s
7:	learn: 1.3613021	test: 1.6840772	best: 1.6840772 (7)	total: 618ms	remaining: 53.4s
8:	learn: 1.3104070	test: 1.6481175	best: 1.6481175 (8)	total: 678ms	remaining: 52.1s
9:	learn: 1.2652794	test: 1.6225682	best: 1.6225682 (9)	total: 742ms	remaining: 51.2s
10:	learn: 1.2250383	test: 1.5885188	best: 1.5885188 (10)	total: 798ms	remaining: 50s
11:	learn: 1.1888244	test: 1.5614442	best: 1.5614442 (1

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   47.2s remaining:    0.0s


0:	learn: 1.9913821	test: 2.0931532	best: 2.0931532 (0)	total: 73.8ms	remaining: 51.6s
1:	learn: 1.8448620	test: 2.0124358	best: 2.0124358 (1)	total: 139ms	remaining: 48.6s
2:	learn: 1.7337577	test: 1.9451967	best: 1.9451967 (2)	total: 204ms	remaining: 47.4s
3:	learn: 1.6343790	test: 1.8877859	best: 1.8877859 (3)	total: 268ms	remaining: 46.7s
4:	learn: 1.5559707	test: 1.8383782	best: 1.8383782 (4)	total: 340ms	remaining: 47.3s
5:	learn: 1.4829710	test: 1.7796523	best: 1.7796523 (5)	total: 396ms	remaining: 45.8s
6:	learn: 1.4219705	test: 1.7432133	best: 1.7432133 (6)	total: 461ms	remaining: 45.7s
7:	learn: 1.3676854	test: 1.7101307	best: 1.7101307 (7)	total: 529ms	remaining: 45.8s
8:	learn: 1.3187970	test: 1.6809587	best: 1.6809587 (8)	total: 602ms	remaining: 46.2s
9:	learn: 1.2744242	test: 1.6465855	best: 1.6465855 (9)	total: 657ms	remaining: 45.3s
10:	learn: 1.2341028	test: 1.6237336	best: 1.6237336 (10)	total: 724ms	remaining: 45.3s
11:	learn: 1.1984124	test: 1.6038860	best: 1.603886

...

0:	learn: 1.8092792	test: 2.0049537	best: 2.0049537 (0)	total: 75.2ms	remaining: 52.5s
1:	learn: 1.6117393	test: 1.8788003	best: 1.8788003 (1)	total: 141ms	remaining: 49.4s
2:	learn: 1.4692040	test: 1.7880351	best: 1.7880351 (2)	total: 208ms	remaining: 48.2s
3:	learn: 1.3579570	test: 1.7221477	best: 1.7221477 (3)	total: 273ms	remaining: 47.6s
4:	learn: 1.2692557	test: 1.6655544	best: 1.6655544 (4)	total: 350ms	remaining: 48.6s
5:	learn: 1.1942538	test: 1.6203807	best: 1.6203807 (5)	total: 418ms	remaining: 48.3s
6:	learn: 1.1330840	test: 1.5812270	best: 1.5812270 (6)	total: 484ms	remaining: 47.9s
7:	learn: 1.0795272	test: 1.5548310	best: 1.5548310 (7)	total: 556ms	remaining: 48.1s
8:	learn: 1.0326208	test: 1.5243804	best: 1.5243804 (8)	total: 622ms	remaining: 47.7s
9:	learn: 0.9869976	test: 1.5030778	best: 1.5030778 (9)	total: 686ms	remaining: 47.3s
10:	learn: 0.9486241	test: 1.4815016	best: 1.4815016 (10)	total: 749ms	remaining: 46.9s
11:	learn: 0.9135440	test: 1.4659235	best: 1.465923

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  6.4min remaining:    0.0s


0:	learn: 1.8083235	test: 2.0073924	best: 2.0073924 (0)	total: 67ms	remaining: 46.8s
1:	learn: 1.6114533	test: 1.8772748	best: 1.8772748 (1)	total: 144ms	remaining: 50.2s
2:	learn: 1.4657009	test: 1.7659617	best: 1.7659617 (2)	total: 205ms	remaining: 47.6s
3:	learn: 1.3592834	test: 1.7000563	best: 1.7000563 (3)	total: 281ms	remaining: 49s
4:	learn: 1.2741846	test: 1.6505182	best: 1.6505182 (4)	total: 348ms	remaining: 48.4s
5:	learn: 1.2000026	test: 1.6070152	best: 1.6070152 (5)	total: 413ms	remaining: 47.8s
6:	learn: 1.1387404	test: 1.5708526	best: 1.5708526 (6)	total: 478ms	remaining: 47.3s
7:	learn: 1.0853914	test: 1.5333277	best: 1.5333277 (7)	total: 558ms	remaining: 48.3s
8:	learn: 1.0407704	test: 1.5063241	best: 1.5063241 (8)	total: 617ms	remaining: 47.4s
9:	learn: 0.9942112	test: 1.4893766	best: 1.4893766 (9)	total: 681ms	remaining: 47s
10:	learn: 0.9560114	test: 1.4747399	best: 1.4747399 (10)	total: 746ms	remaining: 46.7s
11:	learn: 0.9220032	test: 1.4603390	best: 1.4603390 (11)

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  7.2min remaining:    0.0s


Streaming output truncated to the last 5000 lines.
303:	learn: 0.8300364	test: 1.4121700	best: 1.4121700 (303)	total: 21s	remaining: 27.4s
304:	learn: 0.8287386	test: 1.4116690	best: 1.4116690 (304)	total: 21.1s	remaining: 27.3s
305:	learn: 0.8273255	test: 1.4111676	best: 1.4111676 (305)	total: 21.1s	remaining: 27.2s
306:	learn: 0.8261512	test: 1.4105631	best: 1.4105631 (306)	total: 21.2s	remaining: 27.2s
307:	learn: 0.8249346	test: 1.4101492	best: 1.4101492 (307)	total: 21.3s	remaining: 27.1s
308:	learn: 0.8237704	test: 1.4095673	best: 1.4095673 (308)	total: 21.3s	remaining: 27s
309:	learn: 0.8223613	test: 1.4091453	best: 1.4091453 (309)	total: 21.4s	remaining: 26.9s
310:	learn: 0.8210182	test: 1.4087855	best: 1.4087855 (310)	total: 21.5s	remaining: 26.9s
311:	learn: 0.8196048	test: 1.4083754	best: 1.4083754 (311)	total: 21.5s	remaining: 26.8s
312:	learn: 0.8182213	test: 1.4078993	best: 1.4078993 (312)	total: 21.6s	remaining: 26.7s
313:	learn: 0.8169018	test: 1.4075379	best: 1.4075379

In [ ]:
print("Best Params:", Grid_CBC.best_params_)

### Save model and load

In [ ]:
# import joblib
# joblib.dump(tune_search_model, 'genetic_model.pkl')

In [ ]:
# trained_model = joblib.load('genetic_model.pkl')

### Create predictions

In [ ]:
# predictions_test = trained_model.predict(X_test_scaled)

In [ ]:
predictions_test = Grid_CBC.predict(X_test_scaled)
predictions_test

In [ ]:
# for catboost

predictions_tests = []
for x in predictions_test:
  predictions_tests.append(x[0])

### Create Submission file

In [ ]:
predictions_genetic_disorder_test = []
predictions_disorder_subclass_test = []

for myString in predictions_tests:
  genetic_disorder, disorder_subclass = myString.split('<->')
  predictions_genetic_disorder_test.append(genetic_disorder) 
  predictions_disorder_subclass_test.append(disorder_subclass)

print("length genetic_disorder list:", len(predictions_genetic_disorder_test))
print("length disorder_subclass list:", len(predictions_disorder_subclass_test))

In [ ]:
read = pd.read_csv(gDrivePath + 'test.csv')
read.shape

In [ ]:
submission = pd.DataFrame({
        "Patient Id": read["Patient Id"],
        "Genetic Disorder": predictions_genetic_disorder_test,
        "Disorder Subclass": predictions_disorder_subclass_test,
    })

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)